In [21]:
%serialconnect --port=COM15

Connecting to --port=COM15 --baud=115200 
Ready.


In [22]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.19.1-529-gdaf16826e-dirty on 2022-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.19.1-529-gdaf16826e-dirty on 2022-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [23]:
from tools import CANMotorController

In [24]:
from esp32 import CAN
# TX,GPIO15,PIN 21
# RX,GPIO13,PIN 20
bus = CAN(0, tx=15, rx=13, baudrate=1000000, mode=CAN.NORMAL)

In [25]:
print(bus)

CAN(tx=15, rx=13, baudrate=1000000kb, mode=NORMAL, loopback=0)


In [26]:
motor = CANMotorController(bus, motor_id=127, main_can_id=254)

## Jog
单个参数写入（通信类型 18）

In [27]:
jog_vel = -15  # rad/s
uint_value = motor._float_to_uint(jog_vel, motor.V_MIN, motor.V_MAX, 16)
jog_vel_bytes = motor.format_data(data=[uint_value], format="u16", type="encode")[:2][::-1]

data1 = [0x05, 0x70, 0x00, 0x00, 0x07, 0x01] + jog_vel_bytes
motor.clear_can_rx()
received_msg_data, received_msg_arbitration_id = motor.send_receive_can_message(
    cmd_mode=motor.CmdModes.SINGLE_PARAM_WRITE, data2=motor.MAIN_CAN_ID, data1=data1
)
motor.parse_received_msg(received_msg_data, received_msg_arbitration_id)

Sent message with ID 0x1200fe7f, data: [5, 112, 0, 0, 7, 1, 63, 255]
Received message with ID 0x2807ffe
pos:0.10 rad, vel:0.03 rad/s


## 停止

In [28]:
motor.disable()

Sent message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:-12.50 rad, vel:-15.01 rad/s


In [10]:
motor.set_0_pos()

Sent message with ID 0x600fe7f, data: [1]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:0.12 rad/s


## 读取速度和位置
- 发送参数：单个参数写⼊（通信类型18），写入地址0x7018（电流限制）值为 27A
- 读取返回值：电机反馈数据（通信类型2），Byte0,1: 当前⻆度，Byte2,3:当前⻆速度

In [11]:
motor.write_single_param(param_name="limit_cur", value=27)

Sent message with ID 0x1200fe7f, data: [24, 112, 0, 0, 0, 0, 216, 65]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:-0.00 rad/s


# 位置模式
## 发送电机模式参数写入命令（通信类型 18）
设置 `runmode` 参数为 1
- index(Byte0~1): `run_mode`，0x7005
- value(Byte4~7): 1(位置模式)

In [12]:
motor.write_single_param("run_mode", value=1)

Sent message with ID 0x1200fe7f, data: [5, 112, 0, 0, 1, 0, 0, 0]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:-0.03 rad/s


In [13]:
motor.enable()

Sent message with ID 0x300fe7f, data: []
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:0.06 rad/s


## 最大速度：发送电机模式参数写入命令（通信类型 18）
设置 `limit_spd` 参数为预设最大速度指令
- index(Byte0~1): `limit_spd`, 0x7017
- value(Byte4~7): `float` [0,30]rad/s

In [14]:
motor.write_single_param("limit_spd", value=10)

Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 32, 65]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.01 rad/s


## 目标位置：发送电机模式参数写入命令（通信类型 18）
设置 `loc_ref` 参数为预设位置指令
- index(Byte0~1): `loc_ref`, 0x7016
- value(Byte4~7): `float` rad

In [15]:
motor.write_single_param("loc_ref", value=3.14)

Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 195, 245, 72, 64]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.10 rad/s


In [16]:
import time
# 定义音符到电机参数的映射，为后面的音符分配更高的转速值
note_to_speed = {"C": 6.6, "D": 6.8, "E": 7.3, "F": 8, "G": 9, "A": 10, "B": 13}

# 定义小星星的音符和时长
# 完整定义小星星的音符和时长
notes = ["C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C",
         "G", "G", "F", "F", "E", "E", "D",
         "G", "G", "F", "F", "E", "E", "D",
         "C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C"]

durations = [1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2]  # 以四分音符为单位

x_speed = 1.5 # 速度倍数
x_duration = 0.4 # 间隔倍数，如果小于1，就是减小时间间隔


# 初始化电机的当前位置和方向标志
current_loc_ref = 0
direction_flag = 1  # 1表示正方向，-1表示反方向

motor.disable() #
motor.set_motor_position_control(limit_spd=10, loc_ref=0)
motor.set_0_pos()  # 当前0位
motor.enable()

# 播放小星星
for note, duration in zip(notes, durations):
    duration = duration * x_duration
    
    # 计算电机的转速和目标位置
    limit_spd = note_to_speed[note] * x_speed
    loc_ref = current_loc_ref + (direction_flag * limit_spd * duration * 1.5)

    # 使用电机参数设置函数
    motor.set_motor_position_control(limit_spd=limit_spd, loc_ref=loc_ref)

    # 更新电机的当前位置
    current_loc_ref = loc_ref

    # 反转方向标志
#     direction_flag *= -1

    # 休眠一段时间以模拟音符的时长（这里假设一个四分音符的时长是1秒）
    time.sleep(duration * x_speed)
disable()

Sent message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:3.14 rad, vel:-0.03 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 32, 65]
Received message with ID 0x2007ffe
pos:3.14 rad, vel:0.06 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:3.14 rad, vel:-0.00 rad/s
Sent message with ID 0x600fe7f, data: [1]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:-0.07 rad/s
Sent message with ID 0x300fe7f, data: []
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.01 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.04 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 122, 20, 190, 64]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.02 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:5.

pos:12.50 rad, vel:5.53 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:5.87 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 13, 215, 103, 67]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:4.98 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:5.57 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 177, 199, 109, 67]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:4.81 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 88, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:5.67 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 75, 225, 117, 67]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:4.85 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 88, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:7.61 rad/s
Sent messag

Traceback (most recent call last):
  File "<stdin>", line 53, in <module>
NameError: name 'disable' isn't defined


In [17]:
motor.set_0_pos()
motor.disable()

Sent message with ID 0x600fe7f, data: [1]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:0.23 rad/s
Sent message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:0.00 rad, vel:1.95 rad/s


In [18]:
motor.enable()

Sent message with ID 0x300fe7f, data: []
Received message with ID 0x2807ffe
pos:0.10 rad, vel:0.03 rad/s


In [19]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.19.1-529-gdaf16826e-dirty on 2022-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.19.1-529-gdaf16826e-dirty on 2022-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [20]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.19.1-529-gdaf16826e-dirty on 2022-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x21d8f086ca0, open=True>(port='COM15', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
